In [2]:
import argparse
from collections import defaultdict
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
import argparse
import os
from pathlib import Path
import pickle
import timeit
import platform
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from scipy import spatial
from model_E import GOODLE_E
import os

In [3]:
def cal_dist(positions, accessible_R, ks, Rx, Ry, Rz):
    num = len(positions)
    tmp = np.repeat(np.array(positions), 4, axis=0)
    tmp_x, tmp_y, tmp_z = np.zeros(tmp.shape), np.zeros(tmp.shape), np.zeros(tmp.shape)
    tmp_x[:, 0] = tmp[:, 0]
    tmp_y[:, 1] = tmp[:, 1]
    tmp_z[:, 2] = tmp[:, 2]
    tmp_kx = np.repeat(np.expand_dims(np.repeat(ks[:, 0].unsqueeze(1).cpu().numpy(), tmp.shape[0], axis=1), 1),
                       tmp.shape[0], axis=1)
    tmp_ky = np.repeat(np.expand_dims(np.repeat(ks[:, 1].unsqueeze(1).cpu().numpy(), tmp.shape[0], axis=1), 1),
                       tmp.shape[0], axis=1)
    tmp_kz = np.repeat(np.expand_dims(np.repeat(ks[:, 2].unsqueeze(1).cpu().numpy(), tmp.shape[0], axis=1), 1),
                       tmp.shape[0], axis=1)
    distance_matrices, dist_xs, dist_ys, dist_zs, kRs = [], [], [], [], []
    ori_dist = spatial.distance_matrix(tmp, tmp)
    for R in accessible_R:
        dist_R = spatial.distance_matrix(tmp, tmp - R)
        distance_matrices.append(dist_R)
        dist_x = (np.repeat(np.expand_dims(tmp_x[:, 0], axis=1), 4 * num, axis=1) -
                  np.repeat(np.expand_dims(tmp_x[:, 0], axis=0), 4 * num, axis=0))
        dist_xs.append(dist_x)
        dist_y = (np.repeat(np.expand_dims(tmp_y[:, 1], axis=1), 4 * num, axis=1) -
                  np.repeat(np.expand_dims(tmp_y[:, 1], axis=0), 4 * num, axis=0))
        dist_ys.append(dist_y)
        dist_z = (np.repeat(np.expand_dims(tmp_z[:, 2], axis=1), 4 * num, axis=1) -
                  np.repeat(np.expand_dims(tmp_z[:, 2], axis=0), 4 * num, axis=0))
        dist_zs.append(dist_z)
        kR = tmp_kx * dist_x / Rx + tmp_ky * dist_y / Ry + tmp_kz * dist_z / Rz
        kRs.append(kR * (-np.pi))
    return distance_matrices, dist_xs, dist_ys, dist_zs, kRs, ori_dist


def read_file(filename):
    with open(filename) as file:
        data = file.readlines()
    # print(filename)
    flag = True
    # read position from file
    Rx, Ry, Rz, alpha, beta, gamma = data[0].split()[:]
    Rx, Ry, Rz, alpha, beta, gamma = float(Rx), float(Ry), float(Rz), np.deg2rad(float(alpha)), np.deg2rad(float(beta)), np.deg2rad(float(gamma))
    R2 = np.array([1, 0, 0])
    R3 = np.array([np.cos(gamma), np.sin(gamma), 0])
    tmp_x = np.cos(beta)
    tmp_y = (np.cos(alpha) - tmp_x * np.cos(gamma)) / np.sin(gamma)
    tmp_z = np.sqrt(1 - tmp_x**2 - tmp_y ** 2)
    R4 = np.array([tmp_x, tmp_y, tmp_z])

    accessible_R = []
    # motice that the original lattice vector has been normalized.
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            for k in [-1, 0, 1]:
                accessible_R.append(R2 * i + R3 * j + R4 * k)
    positions = []
    for line in range(1, 30):
        info = data[line].split()
        if info[0] != 'C':
            break
        atom, x, y, z = data[line].split()
        x, y, z = float(x), float(y), float(z)
        positions.append(np.array([x, y, z]))
    ks = [[0,0,0]]
    true_bands = [[float(data[line])]]
    return ks, true_bands, positions, accessible_R, Rx, Ry, Rz

In [5]:
torch.set_default_tensor_type(torch.DoubleTensor)
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = torch.load("model_Crystalline_E.pt").to(device)

In [12]:
file_name = "pre_processed/10_195_19-1753.txt.npy"
# do the preprocessing for file 10_195_19-1753.txt then you can obtain this preprocessed file
# related codes can be found in preprocess.py
# notice that the optimized structure is required
positions, distance_matrices, ori_dist, dist_xs, dist_ys, dist_zs, kRs, accessible_R, ks, true_bands = np.load(file_name, allow_pickle=True)
prediction = model.forward(torch.tensor(positions, device=device),
                                  distance_matrices,
                                  ori_dist,
                                  dist_xs,
                                  dist_ys,
                                  dist_zs,
                                  kRs,
                                  torch.tensor(accessible_R,device=device),
                                  torch.tensor(ks, device=device))
true = torch.tensor(true_bands, device=device)
print(f"the predicted result is {prediction.cpu().detach().numpy()}")
print(f"the error is {prediction-true}")

the predicted result is [[-77.45569915]]
the error is tensor([[0.2199]], device='cuda:0', grad_fn=<SubBackward0>)


In [7]:
train_sets = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
for num in train_sets:
    all_names = os.listdir("all_data/"+str(num))
    count = 0
    total_loss = 0
    for name in all_names:
        count += 1
        if count == 30:
            count = 0
            break
        positions, distance_matrices, ori_dist, dist_xs, dist_ys, dist_zs, kRs, accessible_R, ks, true_bands = np.load("pre_processed/" + name + ".npy", allow_pickle=True)
        predict_bands = model.forward(torch.tensor(positions, device=device),
                                          distance_matrices,
                                          ori_dist,
                                          dist_xs,
                                          dist_ys,
                                          dist_zs,
                                          kRs,
                                          torch.tensor(accessible_R,device=device),
                                          torch.tensor(ks, device=device))
        true_bands = torch.tensor(true_bands, device=device)
        print(f"{name}, {predict_bands.cpu().detach().numpy()}, {true_bands.cpu().detach().numpy()}, {(true_bands-predict_bands).cpu().detach().numpy()}")

10_195_19-1753.txt, [[-77.45569915]], [[-77.67556796]], [[-0.21986881]]
10_195_20-1956.txt, [[-76.44272681]], [[-77.67556796]], [[-1.23284115]]
10_200_52-565.txt, [[-55.55098215]], [[-62.96453404]], [[-7.41355189]]
10_200_53-378.txt, [[-53.51487885]], [[-62.96453345]], [[-9.4496546]]
10_200_56-1656.txt, [[-57.43045908]], [[-63.29161437]], [[-5.86115529]]
10_200_57-1451.txt, [[-53.14611268]], [[-63.29161435]], [[-10.14550167]]
10_200_60-1609.txt, [[-49.81190896]], [[-63.291615]], [[-13.47970604]]
10_200_61-1498.txt, [[-64.84469123]], [[-63.291615]], [[1.55307623]]
10_200_64-518.txt, [[-60.99912478]], [[-62.9647388]], [[-1.96561402]]
10_200_65-421.txt, [[-62.09599463]], [[-62.9647388]], [[-0.86874417]]
10_208_82-1748.txt, [[-79.37792865]], [[-83.60076837]], [[-4.22283972]]
10_208_83-1973.txt, [[-73.01594409]], [[-83.60076837]], [[-10.58482428]]
10_208_85-1780.txt, [[-73.31832914]], [[-83.60076837]], [[-10.28243923]]
10_208_86-638.txt, [[-60.00648273]], [[-83.60076837]], [[-23.59428564]]


14_200_115-1798.txt, [[-83.81901319]], [[-92.09602694]], [[-8.27701375]]
14_200_116-696.txt, [[-71.5518035]], [[-67.30496612]], [[4.24683738]]
14_200_119-829.txt, [[-94.20013876]], [[-95.54972488]], [[-1.34958612]]
14_200_120-628.txt, [[-105.50971096]], [[-95.54972507]], [[9.95998589]]
14_208_144-1399.txt, [[-91.10862547]], [[-102.82589119]], [[-11.71726572]]
14_208_145-1274.txt, [[-92.55493598]], [[-102.82589119]], [[-10.27095521]]
14_208_152-1149.txt, [[-80.42946711]], [[-86.6542221]], [[-6.22475499]]
14_208_153-943.txt, [[-74.96567872]], [[-86.6542221]], [[-11.68854338]]
14_215_66-2091.txt, [[-90.94960427]], [[-92.69643377]], [[-1.7468295]]
14_215_70-3.txt, [[-94.11531968]], [[-83.23666532]], [[10.87865436]]
14_215_72-1420.txt, [[-84.80048259]], [[-89.20873624]], [[-4.40825365]]
15_195_1-795.txt, [[-91.6311374]], [[-108.63274532]], [[-17.00160792]]
15_195_11-363.txt, [[-92.07146021]], [[-101.15052268]], [[-9.07906247]]
15_195_12-1570.txt, [[-88.73883096]], [[-101.15052268]], [[-12.4

KeyboardInterrupt: 